In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import datetime
import time
import math
import warnings
warnings.filterwarnings("ignore")
import glob

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (25,10)
mpl.rcParams['axes.grid'] = False

In [ ]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization, UtilityFunction
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Mohammad-Kaosain-Akbar/NILM-two-years-dataset/main/Data_Greener_all.csv')
df['Time']= pd.to_datetime(df['Time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Time               43824 non-null  datetime64[ns]
 1   Ventilation        43824 non-null  float64       
 2   Sokets plug        43824 non-null  float64       
 3   Lighting           43824 non-null  float64       
 4   Other electricity  43824 non-null  float64       
 5   Cooling            43824 non-null  float64       
 6   Heating            43824 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 2.3 MB


In [ ]:
df

,Time,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating
0,2017-01-03 13:00:00,27.4,41.8,53.7,89.6,18.60,498.0
1,2017-01-03 14:00:00,21.6,37.5,50.9,64.6,61.90,500.0
2,2017-01-03 15:00:00,18.5,37.4,60.1,64.2,16.50,480.0
3,2017-01-03 16:00:00,29.7,38.0,52.6,43.2,19.80,390.0
4,2017-01-03 17:00:00,16.7,38.1,56.7,42.4,15.20,373.0
...,...,...,...,...,...,...,...
43819,2022-01-03 08:00:00,59.3,17.9,16.6,115.0,12.70,344.0
43820,2022-01-03 09:00:00,71.9,20.6,33.1,149.0,11.90,399.0
43821,2022-01-03 10:00:00,90.5,30.9,47.8,157.0,12.90,397.0
43822,2022-01-03 11:00:00,94.0,33.9,52.2,155.0,15.40,345.0


In [ ]:
Total = df["Ventilation"] + df["Sokets plug"] + df["Lighting"] + df["Other electricity"] + df["Cooling"] + df["Heating"]
# https://www.kite.com/python/answers/how-to-sum-two-columns-in-a-pandas-dataframe-in-python

df["Total"] = Total
df

,Time,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating,Total
0,2017-01-03 13:00:00,27.4,41.8,53.7,89.6,18.60,498.0,729.10
1,2017-01-03 14:00:00,21.6,37.5,50.9,64.6,61.90,500.0,736.50
2,2017-01-03 15:00:00,18.5,37.4,60.1,64.2,16.50,480.0,676.70
3,2017-01-03 16:00:00,29.7,38.0,52.6,43.2,19.80,390.0,573.30
4,2017-01-03 17:00:00,16.7,38.1,56.7,42.4,15.20,373.0,542.10
...,...,...,...,...,...,...,...,...
43819,2022-01-03 08:00:00,59.3,17.9,16.6,115.0,12.70,344.0,565.50
43820,2022-01-03 09:00:00,71.9,20.6,33.1,149.0,11.90,399.0,685.50
43821,2022-01-03 10:00:00,90.5,30.9,47.8,157.0,12.90,397.0,736.10
43822,2022-01-03 11:00:00,94.0,33.9,52.2,155.0,15.40,345.0,695.50


In [ ]:
df['Year'] = df['Time'].dt.year
df['Month']=df['Time'].dt.month
df['Quarter_of_year']=df['Time'].dt.quarter
df['Day_of_week']=df['Time'].dt.day_of_week
df['Week_of_year']=df['Time'].dt.week
df['Day']=df['Time'].dt.day
df['Hour']=df['Time'].dt.hour

df.drop('Time', inplace=True, axis=1)
df

,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating,Total,Year,Month,Quarter_of_year,Day_of_week,Week_of_year,Day,Hour
0,27.4,41.8,53.7,89.6,18.60,498.0,729.10,2017,1,1,1,1,3,13
1,21.6,37.5,50.9,64.6,61.90,500.0,736.50,2017,1,1,1,1,3,14
2,18.5,37.4,60.1,64.2,16.50,480.0,676.70,2017,1,1,1,1,3,15
3,29.7,38.0,52.6,43.2,19.80,390.0,573.30,2017,1,1,1,1,3,16
4,16.7,38.1,56.7,42.4,15.20,373.0,542.10,2017,1,1,1,1,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,59.3,17.9,16.6,115.0,12.70,344.0,565.50,2022,1,1,0,1,3,8
43820,71.9,20.6,33.1,149.0,11.90,399.0,685.50,2022,1,1,0,1,3,9
43821,90.5,30.9,47.8,157.0,12.90,397.0,736.10,2022,1,1,0,1,3,10
43822,94.0,33.9,52.2,155.0,15.40,345.0,695.50,2022,1,1,0,1,3,11


In [ ]:
# df.iloc[df['Ventilation'].idxmax()]

In [ ]:
df_train = df.loc[(df['Year'].isin([2017,2018]))]
df_test = df.loc[(df['Year'].isin([2019]))]

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_train

,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating,Total,Year,Month,Quarter_of_year,Day_of_week,Week_of_year,Day,Hour
0,27.4,41.8,53.70,89.60,18.60,498.0,729.10,2017,1,1,1,1,3,13
1,21.6,37.5,50.90,64.60,61.90,500.0,736.50,2017,1,1,1,1,3,14
2,18.5,37.4,60.10,64.20,16.50,480.0,676.70,2017,1,1,1,1,3,15
3,29.7,38.0,52.60,43.20,19.80,390.0,573.30,2017,1,1,1,1,3,16
4,16.7,38.1,56.70,42.40,15.20,373.0,542.10,2017,1,1,1,1,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17454,0.0,11.7,7.50,11.60,8.69,250.0,289.49,2018,12,4,0,1,31,19
17455,0.0,10.9,9.38,10.50,8.75,234.0,273.53,2018,12,4,0,1,31,20
17456,0.0,12.0,8.06,7.50,10.40,233.0,270.96,2018,12,4,0,1,31,21
17457,0.0,10.3,6.75,9.63,10.90,234.0,271.58,2018,12,4,0,1,31,22


In [ ]:
df_test

,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating,Total,Year,Month,Quarter_of_year,Day_of_week,Week_of_year,Day,Hour
0,0.00,9.44,1.000,13.00,8.84,250.0,282.280,2019,1,1,1,1,1,0
1,0.00,11.30,2.310,8.34,9.78,217.0,248.730,2019,1,1,1,1,1,1
2,0.00,10.30,1.690,11.60,8.75,234.0,266.340,2019,1,1,1,1,1,2
3,4.50,10.50,0.375,7.56,9.06,233.0,264.995,2019,1,1,1,1,1,3
4,4.44,9.31,0.625,8.88,8.75,267.0,299.005,2019,1,1,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,10.30,11.30,13.600,458.00,0.00,222.0,715.200,2019,12,4,1,1,31,19
8756,9.50,11.90,13.400,461.00,0.00,206.0,701.800,2019,12,4,1,1,31,20
8757,8.25,10.10,10.100,458.00,0.00,205.0,691.450,2019,12,4,1,1,31,21
8758,10.50,10.40,10.600,457.00,0.00,222.0,710.500,2019,12,4,1,1,31,22


In [ ]:
# X_train1 = df_train[['Total','Year', 'Month', 'Day', 'Hour']].values
# y_train1 = df_train[['Heating']].values
# X_test1 = df_test[['Total','Year', 'Month', 'Day', 'Hour']].values
# y_test1 = df_test[['Heating']].values
# print(X_train1.shape, y_train1.shape, X_test1.shape, y_test1.shape)

In [ ]:
X_train1 = df_train[['Total','Year', 'Month', 'Day', 'Hour']].values
y_train1 = df_train[['Ventilation','Sokets plug', 'Lighting', 'Other electricity', 'Cooling', 'Heating']].values
X_test1 = df_test[['Total','Year', 'Month', 'Day', 'Hour']].values
y_test1 = df_test[['Ventilation','Sokets plug', 'Lighting', 'Other electricity', 'Cooling', 'Heating']].values
print(X_train1.shape, y_train1.shape, X_test1.shape, y_test1.shape)

(17459, 5) (17459, 6) (8760, 5) (8760, 6)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train1 = sc.fit_transform(X_train1)
X_test1 = sc.fit_transform(X_test1)

y_train1 = sc.fit_transform(y_train1)
y_test1 = sc.fit_transform(y_test1)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import RANSACRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor


from sklearn.metrics import r2_score

In [ ]:
model = MultiOutputRegressor(GradientBoostingRegressor())
model.fit(X_train1, y_train1)
pred = model.predict(X_test1)
   
r2 = r2_score(y_test1,pred) 
print(r2)

0.12044949469978113


# **Model For the Whole dataset**

In [ ]:

def black_box_function(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

# GRADIENT BOOSTING REGRESSOR

    model1 = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           ))
    model1.fit(X_train1, y_train1)
    pred1 = model1.predict(X_test1)



# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train1)
    pred2 = model2.predict(X_test1)


# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train1)
    pred3 = model3.predict(X_test1)


# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train1)
    pred4 = model4.predict(X_test1)


# LGBM REGRESSOR

    model5 = MultiOutputRegressor(LGBMRegressor(
                                                num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                ))
    


    model5.fit(X_train1, y_train1)
    pred5 = model5.predict(X_test1)


# DECISION TREE REGRESSOR

    model6 = MultiOutputRegressor(DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                ))
    


    model6.fit(X_train1, y_train1)
    pred6 = model6.predict(X_test1)


    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6
    
    r2 = r2_score(y_test1,pred) 
    r2 = round(r2, 3)
    return r2


In [ ]:
# bayes_opt requires this to be a dictionary.
# gp_params = {"alpha": 1e-10}

pbounds = {"learning_rate": [.1, 10],    # Gradient Boosting Regressor
           "min_samples_leaf1": [1, 20],
           "n_estimators1": [1, 20],
           "subsample": [0.1,1],
           "max_depth1": [1,50],
           "max_leaf_nodes1": [2,50],
           
           "n_estimators2": [50,200],   # Random Forest Regressor
           "max_depth2": [1,100],
           "min_samples_leaf2": [1,20],
           "max_leaf_nodes2" : [2,10],

           "n_neighbors" : [5,100],     
           "leaf_size": [30, 60],
           "p": [1,2],

           "min_samples": [2,100],
           "max_trials": [50,200],
           "stop_probability": [0,1],

          #  "boosting_type": ['gbdt', 'dart', 'gross', 'rf'],
           "num_leaves" :[31, 150],
           "max_depth3": [-1, 150],
           "n_estimators3": [50, 150],
           "min_split_gain": [20, 150],
           "min_child_samples": [20, 150],
           
          #  "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
          #  "splitter": ['best', 'random'],
           "max_depth4": [1,150],
           "min_samples_split": [2, 200],
           "min_weight_fraction_leaf": [0.0, 0.5],
           "min_impurity_decrease": [0,100],
           "max_leaf_nodes": [1,100]
          #  "max_features": ['auto', 'sqrt', 'log2']
      
           }

In [ ]:
optimizer = BayesianOptimization(f = black_box_function, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.088    |  59.01    |  5.518    |  48.66    |  71.77    |  104.4    |  33.2     |  97.65    |  2.299    |  4.024    |  115.2    |  121.3    |  19.77    |  86.57    |  19.68    |  4.113    |  120.3    |  21.17    |  0.1933   |  1.839    |  193.5    |  93.61    |  95.15    |  124.6    |  1.866    |  0.1732   

In [ ]:
print(optimizer.max)

{'target': 0.355, 'params': {'leaf_size': 55.902675826129204, 'learning_rate': 1.175002004597234, 'max_depth1': 31.9466747089533, 'max_depth2': 4.778814307960311, 'max_depth3': 71.29694440455272, 'max_depth4': 150.0, 'max_leaf_nodes': 51.20016800805156, 'max_leaf_nodes1': 10.933126843648418, 'max_leaf_nodes2': 5.902552165926819, 'max_trials': 97.93272214589295, 'min_child_samples': 119.48364380935755, 'min_impurity_decrease': 31.07176043589078, 'min_samples': 69.78049665343624, 'min_samples_leaf1': 4.066306895363786, 'min_samples_leaf2': 20.0, 'min_samples_split': 172.41932540678891, 'min_split_gain': 115.70498264799228, 'min_weight_fraction_leaf': 0.3946852491983767, 'n_estimators1': 4.8203662191933505, 'n_estimators2': 99.96968960745929, 'n_estimators3': 54.03121651137377, 'n_neighbors': 42.77140448980997, 'num_leaves': 91.49140388821853, 'p': 1.432710670768478, 'stop_probability': 0.3527672823217628, 'subsample': 0.48625632966728066}}


# **For Appliance 1 (Ventilation)**

In [ ]:

def black_box_function(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

# GRADIENT BOOSTING REGRESSOR

    model1 = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           ))
    model1.fit(X_train1, y_train1)
    pred1 = model1.predict(X_test1)
    vent_pred = pred1[:,[0,]]
    pred1 = np.asarray(vent_pred).reshape(-1)
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train1)
    pred2 = model2.predict(X_test1)
    vent_pred = pred2[:,[0,]]
    pred2 = np.asarray(vent_pred).reshape(-1)

# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train1)
    pred3 = model3.predict(X_test1)
    vent_pred = pred3[:,[0,]]
    pred3 = np.asarray(vent_pred).reshape(-1)


# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train1)
    pred4 = model4.predict(X_test1)
    vent_pred = pred4[:,[0,]]
    pred4 = np.asarray(vent_pred).reshape(-1)


# LGBM REGRESSOR

    model5 = MultiOutputRegressor(LGBMRegressor(
                                                num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                ))
    


    model5.fit(X_train1, y_train1)
    pred5 = model5.predict(X_test1)
    vent_pred = pred5[:,[0,]]
    pred5 = np.asarray(vent_pred).reshape(-1)


# DECISION TREE REGRESSOR

    model6 = MultiOutputRegressor(DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                ))
    


    model6.fit(X_train1, y_train1)
    pred6 = model6.predict(X_test1)
    vent_pred = pred6[:,[0,]]
    pred6 = np.asarray(vent_pred).reshape(-1)


    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[0,]]
    test = np.asarray(test).reshape(-1)


    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [ ]:
# bayes_opt requires this to be a dictionary.
# gp_params = {"alpha": 1e-10}

pbounds = {"learning_rate": [.1, 10],    # Gradient Boosting Regressor
           "min_samples_leaf1": [1, 20],
           "n_estimators1": [1, 20],
           "subsample": [0.1,1],
           "max_depth1": [1,50],
           "max_leaf_nodes1": [2,50],
           
           "n_estimators2": [50,200],   # Random Forest Regressor
           "max_depth2": [1,100],
           "min_samples_leaf2": [1,20],
           "max_leaf_nodes2" : [2,10],

           "n_neighbors" : [5,100],     
           "leaf_size": [30, 60],
           "p": [1,2],

           "min_samples": [2,100],
           "max_trials": [50,200],
           "stop_probability": [0,1],

          #  "boosting_type": ['gbdt', 'dart', 'gross', 'rf'],
           "num_leaves" :[31, 150],
           "max_depth3": [-1, 150],
           "n_estimators3": [50, 150],
           "min_split_gain": [20, 150],
           "min_child_samples": [20, 150],
           
          #  "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
          #  "splitter": ['best', 'random'],
           "max_depth4": [1,150],
           "min_samples_split": [2, 200],
           "min_weight_fraction_leaf": [0.0, 0.5],
           "min_impurity_decrease": [0,100],
           "max_leaf_nodes": [1,100]
          #  "max_features": ['auto', 'sqrt', 'log2']
      
           }

In [ ]:
optimizer = BayesianOptimization(f = black_box_function, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.14     |  59.01    |  5.518    |  48.66    |  71.77    |  104.4    |  33.2     |  97.65    |  2.299    |  4.024    |  115.2    |  121.3    |  19.77    |  86.57    |  19.68    |  4.113    |  120.3    |  21.17    |  0.1933   |  1.839    |  193.5    |  93.61    |  95.15    |  124.6    |  1.866    |  0.1732   

In [ ]:
print(optimizer.max)

{'target': 0.283, 'params': {'leaf_size': 40.95323321746785, 'learning_rate': 0.5102585329794667, 'max_depth1': 2.0977717975049357, 'max_depth2': 49.63262180896442, 'max_depth3': 12.121408942093481, 'max_depth4': 47.88788138496119, 'max_leaf_nodes': 45.901640081672916, 'max_leaf_nodes1': 15.734809522601815, 'max_leaf_nodes2': 9.452019160520557, 'max_trials': 123.37466796099353, 'min_child_samples': 114.13068975100707, 'min_impurity_decrease': 27.66054192665863, 'min_samples': 85.84743829227709, 'min_samples_leaf1': 8.037445954746243, 'min_samples_leaf2': 15.196647969575313, 'min_samples_split': 66.97935612665114, 'min_split_gain': 136.56480740281586, 'min_weight_fraction_leaf': 0.020360004682908606, 'n_estimators1': 8.467128434132313, 'n_estimators2': 142.66145381522983, 'n_estimators3': 111.29117135829225, 'n_neighbors': 24.24645520165834, 'num_leaves': 33.05524794337437, 'p': 1.7186601231476868, 'stop_probability': 0.8343280844435664, 'subsample': 0.9782796680638138}}


# **Appliance 2 (Socket Plugs)**

In [ ]:

def black_box_function(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

# GRADIENT BOOSTING REGRESSOR

    model1 = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           ))
    model1.fit(X_train1, y_train1)
    pred1 = model1.predict(X_test1)
    soc_pred = pred1[:,[1,]]
    pred1 = np.asarray(soc_pred).reshape(-1)
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train1)
    pred2 = model2.predict(X_test1)
    soc_pred = pred2[:,[1,]]
    pred2 = np.asarray(soc_pred).reshape(-1)

# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train1)
    pred3 = model3.predict(X_test1)
    soc_pred = pred3[:,[1,]]
    pred3 = np.asarray(soc_pred).reshape(-1)


# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train1)
    pred4 = model4.predict(X_test1)
    soc_pred = pred4[:,[1,]]
    pred4 = np.asarray(soc_pred).reshape(-1)


# LGBM REGRESSOR

    model5 = MultiOutputRegressor(LGBMRegressor(
                                                num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                ))
    


    model5.fit(X_train1, y_train1)
    pred5 = model5.predict(X_test1)
    soc_pred = pred5[:,[1,]]
    pred5 = np.asarray(soc_pred).reshape(-1)


# DECISION TREE REGRESSOR

    model6 = MultiOutputRegressor(DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                ))
    


    model6.fit(X_train1, y_train1)
    pred6 = model6.predict(X_test1)
    soc_pred = pred6[:,[1,]]
    pred6 = np.asarray(soc_pred).reshape(-1)


    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[1,]]
    test = np.asarray(test).reshape(-1)


    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [ ]:
# bayes_opt requires this to be a dictionary.
# gp_params = {"alpha": 1e-10}

pbounds = {"learning_rate": [.1, 10],    # Gradient Boosting Regressor
           "min_samples_leaf1": [1, 20],
           "n_estimators1": [1, 20],
           "subsample": [0.1,1],
           "max_depth1": [1,50],
           "max_leaf_nodes1": [2,50],
           
           "n_estimators2": [50,200],   # Random Forest Regressor
           "max_depth2": [1,100],
           "min_samples_leaf2": [1,20],
           "max_leaf_nodes2" : [2,10],

           "n_neighbors" : [5,100],     
           "leaf_size": [30, 60],
           "p": [1,2],

           "min_samples": [2,100],
           "max_trials": [50,200],
           "stop_probability": [0,1],

          #  "boosting_type": ['gbdt', 'dart', 'gross', 'rf'],
           "num_leaves" :[31, 150],
           "max_depth3": [-1, 150],
           "n_estimators3": [50, 150],
           "min_split_gain": [20, 150],
           "min_child_samples": [20, 150],
           
          #  "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
          #  "splitter": ['best', 'random'],
           "max_depth4": [1,150],
           "min_samples_split": [2, 200],
           "min_weight_fraction_leaf": [0.0, 0.5],
           "min_impurity_decrease": [0,100],
           "max_leaf_nodes": [1,100]
          #  "max_features": ['auto', 'sqrt', 'log2']
      
           }

In [ ]:
optimizer = BayesianOptimization(f = black_box_function, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -0.083    |  59.01    |  5.518    |  48.66    |  71.77    |  104.4    |  33.2     |  97.65    |  2.299    |  4.024    |  115.2    |  121.3    |  19.77    |  86.57    |  19.68    |  4.113    |  120.3    |  21.17    |  0.1933   |  1.839    |  193.5    |  93.61    |  95.15    |  124.6    |  1.866    |  0.1732   

In [ ]:
print(optimizer.max)

{'target': 0.242, 'params': {'leaf_size': 60.0, 'learning_rate': 0.1, 'max_depth1': 50.0, 'max_depth2': 1.0, 'max_depth3': 11.737449796355437, 'max_depth4': 150.0, 'max_leaf_nodes': 1.0, 'max_leaf_nodes1': 2.0, 'max_leaf_nodes2': 10.0, 'max_trials': 50.0, 'min_child_samples': 150.0, 'min_impurity_decrease': 0.0, 'min_samples': 2.0, 'min_samples_leaf1': 20.0, 'min_samples_leaf2': 20.0, 'min_samples_split': 2.0, 'min_split_gain': 20.0, 'min_weight_fraction_leaf': 0.0, 'n_estimators1': 20.0, 'n_estimators2': 200.0, 'n_estimators3': 149.59095134158264, 'n_neighbors': 5.0, 'num_leaves': 150.0, 'p': 2.0, 'stop_probability': 0.0, 'subsample': 1.0}}


# **Appliance 3 (Lighting)**

In [ ]:

def black_box_function(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

# GRADIENT BOOSTING REGRESSOR

    model1 = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           ))
    model1.fit(X_train1, y_train1)
    pred1 = model1.predict(X_test1)
    light_pred = pred1[:,[2,]]
    pred1 = np.asarray(light_pred).reshape(-1)
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train1)
    pred2 = model2.predict(X_test1)
    light_pred = pred2[:,[2,]]
    pred2 = np.asarray(light_pred).reshape(-1)

# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train1)
    pred3 = model3.predict(X_test1)
    light_pred = pred3[:,[2,]]
    pred3 = np.asarray(light_pred).reshape(-1)


# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train1)
    pred4 = model4.predict(X_test1)
    light_pred = pred4[:,[2,]]
    pred4 = np.asarray(light_pred).reshape(-1)


# LGBM REGRESSOR

    model5 = MultiOutputRegressor(LGBMRegressor(
                                                num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                ))
    


    model5.fit(X_train1, y_train1)
    pred5 = model5.predict(X_test1)
    light_pred = pred5[:,[2,]]
    pred5 = np.asarray(light_pred).reshape(-1)


# DECISION TREE REGRESSOR

    model6 = MultiOutputRegressor(DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                ))
    


    model6.fit(X_train1, y_train1)
    pred6 = model6.predict(X_test1)
    light_pred = pred6[:,[2,]]
    pred6 = np.asarray(light_pred).reshape(-1)


    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[2,]]
    test = np.asarray(test).reshape(-1)


    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [ ]:
# bayes_opt requires this to be a dictionary.
# gp_params = {"alpha": 1e-10}

pbounds = {"learning_rate": [.1, 10],    # Gradient Boosting Regressor
           "min_samples_leaf1": [1, 20],
           "n_estimators1": [1, 20],
           "subsample": [0.1,1],
           "max_depth1": [1,50],
           "max_leaf_nodes1": [2,50],
           
           "n_estimators2": [50,200],   # Random Forest Regressor
           "max_depth2": [1,100],
           "min_samples_leaf2": [1,20],
           "max_leaf_nodes2" : [2,10],

           "n_neighbors" : [5,100],     
           "leaf_size": [30, 60],
           "p": [1,2],

           "min_samples": [2,100],
           "max_trials": [50,200],
           "stop_probability": [0,1],

          #  "boosting_type": ['gbdt', 'dart', 'gross', 'rf'],
           "num_leaves" :[31, 150],
           "max_depth3": [-1, 150],
           "n_estimators3": [50, 150],
           "min_split_gain": [20, 150],
           "min_child_samples": [20, 150],
           
          #  "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
          #  "splitter": ['best', 'random'],
           "max_depth4": [1,150],
           "min_samples_split": [2, 200],
           "min_weight_fraction_leaf": [0.0, 0.5],
           "min_impurity_decrease": [0,100],
           "max_leaf_nodes": [1,100]
          #  "max_features": ['auto', 'sqrt', 'log2']
      
           }

In [ ]:
optimizer = BayesianOptimization(f = black_box_function, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.348    |  59.01    |  5.518    |  48.66    |  71.77    |  104.4    |  33.2     |  97.65    |  2.299    |  4.024    |  115.2    |  121.3    |  19.77    |  86.57    |  19.68    |  4.113    |  120.3    |  21.17    |  0.1933   |  1.839    |  193.5    |  93.61    |  95.15    |  124.6    |  1.866    |  0.1732   

In [ ]:
print(optimizer.max)

{'target': 0.554, 'params': {'leaf_size': 41.550027865320466, 'learning_rate': 2.140849339697943, 'max_depth1': 28.414101942739457, 'max_depth2': 84.10405433376371, 'max_depth3': 116.95765688603004, 'max_depth4': 139.35655430278774, 'max_leaf_nodes': 75.88757953096373, 'max_leaf_nodes1': 39.733443805307985, 'max_leaf_nodes2': 5.21987559465972, 'max_trials': 71.17085268272214, 'min_child_samples': 114.2726926377367, 'min_impurity_decrease': 4.731061744268805, 'min_samples': 18.434990525072784, 'min_samples_leaf1': 7.393845805173756, 'min_samples_leaf2': 8.795560654139532, 'min_samples_split': 46.84703956145956, 'min_split_gain': 122.54083937161175, 'min_weight_fraction_leaf': 0.4583121768532867, 'n_estimators1': 1.5872470237509402, 'n_estimators2': 194.72541427212354, 'n_estimators3': 121.11377136799004, 'n_neighbors': 96.52491261509813, 'num_leaves': 56.79286263677379, 'p': 1.4855402715425376, 'stop_probability': 0.416028793702647, 'subsample': 0.7735805968689351}}


# **Appliance 4 (Other Electricity)**

In [ ]:

def black_box_function(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

# GRADIENT BOOSTING REGRESSOR

    model1 = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           ))
    model1.fit(X_train1, y_train1)
    pred1 = model1.predict(X_test1)
    other_pred = pred1[:,[3,]]
    pred1 = np.asarray(other_pred).reshape(-1)
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train1)
    pred2 = model2.predict(X_test1)
    other_pred = pred2[:,[3,]]
    pred2 = np.asarray(other_pred).reshape(-1)

# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train1)
    pred3 = model3.predict(X_test1)
    other_pred = pred3[:,[3,]]
    pred3 = np.asarray(other_pred).reshape(-1)


# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train1)
    pred4 = model4.predict(X_test1)
    other_pred = pred4[:,[3,]]
    pred4 = np.asarray(other_pred).reshape(-1)


# LGBM REGRESSOR

    model5 = MultiOutputRegressor(LGBMRegressor(
                                                num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                ))
    


    model5.fit(X_train1, y_train1)
    pred5 = model5.predict(X_test1)
    other_pred = pred5[:,[3,]]
    pred5 = np.asarray(other_pred).reshape(-1)


# DECISION TREE REGRESSOR

    model6 = MultiOutputRegressor(DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                ))
    


    model6.fit(X_train1, y_train1)
    pred6 = model6.predict(X_test1)
    other_pred = pred6[:,[3,]]
    pred6 = np.asarray(other_pred).reshape(-1)


    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[3,]]
    test = np.asarray(test).reshape(-1)


    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [ ]:
# bayes_opt requires this to be a dictionary.
# gp_params = {"alpha": 1e-10}

pbounds = {"learning_rate": [.1, 10],    # Gradient Boosting Regressor
           "min_samples_leaf1": [1, 20],
           "n_estimators1": [1, 20],
           "subsample": [0.1,1],
           "max_depth1": [1,50],
           "max_leaf_nodes1": [2,50],
           
           "n_estimators2": [50,200],   # Random Forest Regressor
           "max_depth2": [1,100],
           "min_samples_leaf2": [1,20],
           "max_leaf_nodes2" : [2,10],

           "n_neighbors" : [5,100],     
           "leaf_size": [30, 60],
           "p": [1,2],

           "min_samples": [2,100],
           "max_trials": [50,200],
           "stop_probability": [0,1],

          #  "boosting_type": ['gbdt', 'dart', 'gross', 'rf'],
           "num_leaves" :[31, 150],
           "max_depth3": [-1, 150],
           "n_estimators3": [50, 150],
           "min_split_gain": [20, 150],
           "min_child_samples": [20, 150],
           
          #  "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
          #  "splitter": ['best', 'random'],
           "max_depth4": [1,150],
           "min_samples_split": [2, 200],
           "min_weight_fraction_leaf": [0.0, 0.5],
           "min_impurity_decrease": [0,100],
           "max_leaf_nodes": [1,100]
          #  "max_features": ['auto', 'sqrt', 'log2']
      
           }

In [ ]:
optimizer = BayesianOptimization(f = black_box_function, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -0.398    |  59.01    |  5.518    |  48.66    |  71.77    |  104.4    |  33.2     |  97.65    |  2.299    |  4.024    |  115.2    |  121.3    |  19.77    |  86.57    |  19.68    |  4.113    |  120.3    |  21.17    |  0.1933   |  1.839    |  193.5    |  93.61    |  95.15    |  124.6    |  1.866    |  0.1732   

In [ ]:
print(optimizer.max)

{'target': 0.339, 'params': {'leaf_size': 60.0, 'learning_rate': 0.1, 'max_depth1': 1.0, 'max_depth2': 100.0, 'max_depth3': -1.0, 'max_depth4': 150.0, 'max_leaf_nodes': 1.0, 'max_leaf_nodes1': 50.0, 'max_leaf_nodes2': 10.0, 'max_trials': 200.0, 'min_child_samples': 150.0, 'min_impurity_decrease': 0.0, 'min_samples': 100.0, 'min_samples_leaf1': 1.0, 'min_samples_leaf2': 1.0, 'min_samples_split': 200.0, 'min_split_gain': 150.0, 'min_weight_fraction_leaf': 0.5, 'n_estimators1': 1.0, 'n_estimators2': 50.0, 'n_estimators3': 150.0, 'n_neighbors': 100.0, 'num_leaves': 150.0, 'p': 2.0, 'stop_probability': 0.0, 'subsample': 1.0}}


# **Appliance 5 (Cooling)**

In [ ]:

def black_box_function(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

# GRADIENT BOOSTING REGRESSOR

    model1 = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           ))
    model1.fit(X_train1, y_train1)
    pred1 = model1.predict(X_test1)
    cool_pred = pred1[:,[4,]]
    pred1 = np.asarray(cool_pred).reshape(-1)
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train1)
    pred2 = model2.predict(X_test1)
    cool_pred = pred2[:,[4,]]
    pred2 = np.asarray(cool_pred).reshape(-1)

# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train1)
    pred3 = model3.predict(X_test1)
    cool_pred = pred3[:,[4,]]
    pred3 = np.asarray(cool_pred).reshape(-1)


# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train1)
    pred4 = model4.predict(X_test1)
    cool_pred = pred4[:,[4,]]
    pred4 = np.asarray(cool_pred).reshape(-1)


# LGBM REGRESSOR

    model5 = MultiOutputRegressor(LGBMRegressor(
                                                num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                ))
    


    model5.fit(X_train1, y_train1)
    pred5 = model5.predict(X_test1)
    cool_pred = pred5[:,[4,]]
    pred5 = np.asarray(cool_pred).reshape(-1)


# DECISION TREE REGRESSOR

    model6 = MultiOutputRegressor(DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                ))
    


    model6.fit(X_train1, y_train1)
    pred6 = model6.predict(X_test1)
    cool_pred = pred6[:,[4,]]
    pred6 = np.asarray(cool_pred).reshape(-1)


    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[4,]]
    test = np.asarray(test).reshape(-1)


    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [ ]:
# bayes_opt requires this to be a dictionary.
# gp_params = {"alpha": 1e-10}

pbounds = {"learning_rate": [.1, 10],    # Gradient Boosting Regressor
           "min_samples_leaf1": [1, 20],
           "n_estimators1": [1, 20],
           "subsample": [0.1,1],
           "max_depth1": [1,50],
           "max_leaf_nodes1": [2,50],
           
           "n_estimators2": [50,200],   # Random Forest Regressor
           "max_depth2": [1,100],
           "min_samples_leaf2": [1,20],
           "max_leaf_nodes2" : [2,10],

           "n_neighbors" : [5,100],     
           "leaf_size": [30, 60],
           "p": [1,2],

           "min_samples": [2,100],
           "max_trials": [50,200],
           "stop_probability": [0,1],

          #  "boosting_type": ['gbdt', 'dart', 'gross', 'rf'],
           "num_leaves" :[31, 150],
           "max_depth3": [-1, 150],
           "n_estimators3": [50, 150],
           "min_split_gain": [20, 150],
           "min_child_samples": [20, 150],
           
          #  "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
          #  "splitter": ['best', 'random'],
           "max_depth4": [1,150],
           "min_samples_split": [2, 200],
           "min_weight_fraction_leaf": [0.0, 0.5],
           "min_impurity_decrease": [0,100],
           "max_leaf_nodes": [1,100]
          #  "max_features": ['auto', 'sqrt', 'log2']
      
           }

In [ ]:
optimizer = BayesianOptimization(f = black_box_function, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.404    |  59.01    |  5.518    |  48.66    |  71.77    |  104.4    |  33.2     |  97.65    |  2.299    |  4.024    |  115.2    |  121.3    |  19.77    |  86.57    |  19.68    |  4.113    |  120.3    |  21.17    |  0.1933   |  1.839    |  193.5    |  93.61    |  95.15    |  124.6    |  1.866    |  0.1732   

In [ ]:
print(optimizer.max)

{'target': 0.775, 'params': {'leaf_size': 50.83783877192613, 'learning_rate': 4.355370925352909, 'max_depth1': 46.62579750201129, 'max_depth2': 82.73760479823859, 'max_depth3': 96.59919978530327, 'max_depth4': 6.634476381314612, 'max_leaf_nodes': 35.996240100716754, 'max_leaf_nodes1': 47.70643569418427, 'max_leaf_nodes2': 7.841190492718921, 'max_trials': 150.53543789921554, 'min_child_samples': 52.05138648711814, 'min_impurity_decrease': 88.20558139643397, 'min_samples': 72.15287200327775, 'min_samples_leaf1': 10.718003236927833, 'min_samples_leaf2': 13.323668680271565, 'min_samples_split': 131.8275161612433, 'min_split_gain': 148.24185706912903, 'min_weight_fraction_leaf': 0.0620737080857196, 'n_estimators1': 1.1732686732910316, 'n_estimators2': 96.72604518002674, 'n_estimators3': 65.18875142720455, 'n_neighbors': 71.29788242768943, 'num_leaves': 66.81515852221737, 'p': 1.962362326515246, 'stop_probability': 0.63903299945697, 'subsample': 0.5331283490670103}}


# **Appliance 6 (Heating)**

In [ ]:

def black_box_function(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

# GRADIENT BOOSTING REGRESSOR

    model1 = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           ))
    model1.fit(X_train1, y_train1)
    pred1 = model1.predict(X_test1)
    heat_pred = pred1[:,[5,]]
    pred1 = np.asarray(heat_pred).reshape(-1)
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train1)
    pred2 = model2.predict(X_test1)
    heat_pred = pred2[:,[5,]]
    pred2 = np.asarray(heat_pred).reshape(-1)

# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train1)
    pred3 = model3.predict(X_test1)
    heat_pred = pred3[:,[5,]]
    pred3 = np.asarray(heat_pred).reshape(-1)


# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train1)
    pred4 = model4.predict(X_test1)
    heat_pred = pred4[:,[5,]]
    pred4 = np.asarray(heat_pred).reshape(-1)


# LGBM REGRESSOR

    model5 = MultiOutputRegressor(LGBMRegressor(
                                                num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                ))
    


    model5.fit(X_train1, y_train1)
    pred5 = model5.predict(X_test1)
    heat_pred = pred5[:,[5,]]
    pred5 = np.asarray(heat_pred).reshape(-1)


# DECISION TREE REGRESSOR

    model6 = MultiOutputRegressor(DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                ))
    


    model6.fit(X_train1, y_train1)
    pred6 = model6.predict(X_test1)
    heat_pred = pred6[:,[5,]]
    pred6 = np.asarray(heat_pred).reshape(-1)


    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[5,]]
    test = np.asarray(test).reshape(-1)


    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [ ]:
# bayes_opt requires this to be a dictionary.
# gp_params = {"alpha": 1e-10}

pbounds = {"learning_rate": [.1, 10],    # Gradient Boosting Regressor
           "min_samples_leaf1": [1, 20],
           "n_estimators1": [1, 20],
           "subsample": [0.1,1],
           "max_depth1": [1,50],
           "max_leaf_nodes1": [2,50],
           
           "n_estimators2": [50,200],   # Random Forest Regressor
           "max_depth2": [1,100],
           "min_samples_leaf2": [1,20],
           "max_leaf_nodes2" : [2,10],

           "n_neighbors" : [5,100],     
           "leaf_size": [30, 60],
           "p": [1,2],

           "min_samples": [2,100],
           "max_trials": [50,200],
           "stop_probability": [0,1],

          #  "boosting_type": ['gbdt', 'dart', 'gross', 'rf'],
           "num_leaves" :[31, 150],
           "max_depth3": [-1, 150],
           "n_estimators3": [50, 150],
           "min_split_gain": [20, 150],
           "min_child_samples": [20, 150],
           
          #  "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
          #  "splitter": ['best', 'random'],
           "max_depth4": [1,150],
           "min_samples_split": [2, 200],
           "min_weight_fraction_leaf": [0.0, 0.5],
           "min_impurity_decrease": [0,100],
           "max_leaf_nodes": [1,100]
          #  "max_features": ['auto', 'sqrt', 'log2']
      
           }

In [ ]:
optimizer = BayesianOptimization(f = black_box_function, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.128    |  59.01    |  5.518    |  48.66    |  71.77    |  104.4    |  33.2     |  97.65    |  2.299    |  4.024    |  115.2    |  121.3    |  19.77    |  86.57    |  19.68    |  4.113    |  120.3    |  21.17    |  0.1933   |  1.839    |  193.5    |  93.61    |  95.15    |  124.6    |  1.866    |  0.1732   

In [ ]:
# print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))

In [ ]:
print(optimizer.max)

{'target': 0.458, 'params': {'leaf_size': 60.0, 'learning_rate': 0.1, 'max_depth1': 50.0, 'max_depth2': 100.0, 'max_depth3': -1.0, 'max_depth4': 150.0, 'max_leaf_nodes': 100.0, 'max_leaf_nodes1': 2.0, 'max_leaf_nodes2': 10.0, 'max_trials': 200.0, 'min_child_samples': 150.0, 'min_impurity_decrease': 0.0, 'min_samples': 100.0, 'min_samples_leaf1': 1.0, 'min_samples_leaf2': 1.0, 'min_samples_split': 2.0, 'min_split_gain': 20.0, 'min_weight_fraction_leaf': 0.0, 'n_estimators1': 20.0, 'n_estimators2': 50.0, 'n_estimators3': 50.0, 'n_neighbors': 100.0, 'num_leaves': 31.0, 'p': 2.0, 'stop_probability': 0.0, 'subsample': 1.0}}
